# Does it matter to start coding after the age of 40?

For some people it is difficult to change their professional field because of the age, especially when it comes to I.T.  
Let's take a look into the results of "Kaggle servey 2021" and find out if the age when you start plays a role in achieving success. 

In this analysis we will compare professional results of people who started coding after the age of 40 to respondents who started erlier and who has never written code. 
So we'll analyze 3 subsets of the data science community aged 40-70+ represented in this survey.
Going forward, one of the findings - it's never too late to start! 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pycountry
import plotly.offline as py

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objs as go
import pycountry
py.init_notebook_mode(connected=True)

data = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')
data.columns = data.iloc[0]
data=data.drop([0])

In [ ]:
# Let's write some support functions

# 1. Calculating the average salary by age group
def mean_salary_by_age(df, title):
    mean_salary = pd.DataFrame(df[['What is your age (# years)?', 'What is your current yearly compensation (approximate $USD)?']].value_counts()).reset_index()
    mean_salary.columns = ['Age', 'Salary', 'Count']
    list_m = []
    for cell in mean_salary.Salary:
        cell = cell.replace(',','')
        cell = cell.replace('$', '')
        cell = cell.replace('>', '')
        cell = cell.replace('+', '')
        cell = cell.split('-')
        if len(cell) == 2:
            a, b = float(cell[0]),float(cell[1])
            m = round((b + a) / 2)
            list_m.append(m)
        else:
            m = float(cell[0])
            list_m.append(m)
    mean_salary[title] = list_m

    for cat in set(mean_salary['Age']):
        mean_salary.loc[(mean_salary['Age'] == cat), title] = round(sum(mean_salary[mean_salary['Age'] == cat]['Count'] * mean_salary[mean_salary['Age'] == cat][title])/mean_salary[mean_salary['Age'] == cat]['Count'].sum())

    return mean_salary.groupby('Age')[title].mean()

# 2. Calculating the average salary by profession
def mean_salary_by_profession(df, title):
    mean_salary = df[['Select the title most similar to your current role (or most recent title if retired): - Selected Choice', 'What is your current yearly compensation (approximate $USD)?']].value_counts().reset_index()
    mean_salary.columns = ['Profession', 'Salary', 'Count']
    list_m = []
    for cell in mean_salary.Salary:
        cell = cell.replace(',','')
        cell = cell.replace('$', '')
        cell = cell.replace('>', '')
        cell = cell.split('-')
        if len(cell) == 2:
            a, b = float(cell[0]),float(cell[1])
            m = round((b + a) / 2)
            list_m.append(m)
        else:
            m = float(cell[0])
            list_m.append(m)
    mean_salary[title] = list_m

    for cat in set(mean_salary['Profession']):
        mean_salary.loc[(mean_salary['Profession'] == cat), title] = round(sum(mean_salary[mean_salary['Profession'] == cat]['Count'] * mean_salary[mean_salary['Profession'] == cat][title])/mean_salary[mean_salary['Profession'] == cat]['Count'].sum())

    return mean_salary.groupby('Profession')[title].mean()

# 3. Calculating the average salary by experience
def mean_salary_by_exp(df, title):
    mean_salary = df[['For how many years have you been writing code and/or programming?', 'What is your current yearly compensation (approximate $USD)?']].value_counts().reset_index()
    mean_salary.columns = ['Coding experience', 'Salary', 'Count']
    list_m = []
    for cell in mean_salary.Salary:
        cell = cell.replace(',','')
        cell = cell.replace('$', '')
        cell = cell.replace('>', '')
        cell = cell.split('-')
        if len(cell) == 2:
            a, b = float(cell[0]),float(cell[1])
            m = round((b + a) / 2)
            list_m.append(m)
        else:
            m = float(cell[0])
            list_m.append(m)
    mean_salary[title] = list_m

    for cat in set(mean_salary['Coding experience']):
        mean_salary.loc[(mean_salary['Coding experience'] == cat), title] = round(sum(mean_salary[mean_salary['Coding experience'] == cat]['Count'] * mean_salary[mean_salary['Coding experience'] == cat][title])/mean_salary[mean_salary['Coding experience'] == cat]['Count'].sum())

    return mean_salary.groupby('Coding experience')[title].mean().reindex(['1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'])

# 4. Graphs
# 4.1
def mean_salary_by_age_plot(df, title):
    mean_salary = pd.DataFrame(df[['What is your age (# years)?', 'What is your current yearly compensation (approximate $USD)?']].value_counts()).reset_index()
    mean_salary.columns = ['Age', 'Salary', 'Count']
    list_m = []
    for cell in mean_salary.Salary:
        cell = cell.replace(',','')
        cell = cell.replace('$', '')
        cell = cell.replace('>', '')
        cell = cell.split('-')
        if len(cell) == 2:
            a, b = float(cell[0]),float(cell[1])
            m = round((b + a) / 2)
            list_m.append(m)
        else:
            m = float(cell[0])
            list_m.append(m)
    mean_salary[title] = list_m

    for cat in set(mean_salary['Age']):
        mean_salary.loc[(mean_salary['Age'] == cat), title] = round(sum(mean_salary[mean_salary['Age'] == cat]['Count'] * mean_salary[mean_salary['Age'] == cat][title])/mean_salary[mean_salary['Age'] == cat]['Count'].sum())

    mean_salary_short = mean_salary.groupby('Age')[title].mean()
    mean_salary_short.plot(kind = 'bar', figsize=(11, 5), title = title)
    
# 4.2 
def mean_salary_by_exp_plot(df, title):
    mean_salary = df[['For how many years have you been writing code and/or programming?', 'What is your current yearly compensation (approximate $USD)?']].value_counts().reset_index()
    mean_salary.columns = ['Coding experience', 'Salary', 'Count']
    list_m = []
    for cell in mean_salary.Salary:
        cell = cell.replace(',','')
        cell = cell.replace('$', '')
        cell = cell.replace('>', '')
        cell = cell.split('-')
        if len(cell) == 2:
            a, b = float(cell[0]),float(cell[1])
            m = round((b + a) / 2)
            list_m.append(m)
        else:
            m = float(cell[0])
            list_m.append(m)
    mean_salary[title] = list_m

    for cat in set(mean_salary['Coding experience']):
        mean_salary.loc[(mean_salary['Coding experience'] == cat), title] = round(sum(mean_salary[mean_salary['Coding experience'] == cat]['Count'] * mean_salary[mean_salary['Coding experience'] == cat][title])/mean_salary[mean_salary['Coding experience'] == cat]['Count'].sum())

    mean_salary_short = mean_salary.groupby('Coding experience')[title].mean().reindex(['1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'])
    mean_salary_short.plot(figsize=(11, 5), title = title)

First of all, let's find out if it's worth it to start coding/programming. The graph below shows us that respondents with experience in coding have higher income than those without coding experience:

In [ ]:
no_code_experience = data[data['For how many years have you been writing code and/or programming?'] == "I have never written code"]
some_code_experience = data[~data.index.isin(no_code_experience.index)]

no_code_exp_short = mean_salary_by_age(no_code_experience, 'No experience')

some_code_exp_short = mean_salary_by_age(some_code_experience, 'Some experience')

compare = pd.concat([some_code_exp_short, no_code_exp_short], axis = 1)
compare.plot(kind = 'bar',  figsize = (11,5), title = 'Mean salary by ages among the data science community')
plt.ylabel('Average yearly compensation, $')
plt.show()

As we can see, the age 70+ is abnormal. Let's take a look into them. 

<!-- It is because there are some outliers in the data -->

In [ ]:
outliers = no_code_experience[no_code_experience['What is your age (# years)?'] == '70+']
outliers[['What is your age (# years)?', 'What is your current yearly compensation (approximate $USD)?']]

There're only 4 respondets. That's why the salary of ">$1 000 000" has a significant impact on the average. 

The difference in salaries dependent on coding experience is characteristic of almost all professions (except Software Engineer and DBA/Database Engineer). For some professions the difference is huge - many times:

In [ ]:
no_code_exp_profession = mean_salary_by_profession(no_code_experience, 'mean_sal_no_exp')
some_code_exp_profession = mean_salary_by_profession(some_code_experience, 'mean_sal_some_exp')
compare_no_some = pd.concat([no_code_exp_profession, some_code_exp_profession], axis = 1)
compare_no_some.plot(kind = 'bar', figsize = (11,5))
plt.ylabel('Average yearly compensation, $')
plt.show()

So the importance of coding experience cannot be overemphasized. And now let's find out if achieving success depends on the age when you start or not.

<!-- it can or cannot be too late to start. -->

## Subsets of respondents 
<!-- who started coding after 40 years (group 1) -->

Let's devide the respondents into three subsamples - who has started coding after 40, before 40 and who has never written code. 

The first subset includes people aged 40-44 years (having 1-3 years of coding experience), 45-49 (having 1-5 years of coding experience), 50-59 (having 1-10 years of coding experience), and 60-70+ (having 1-20+ years of coding experience).

The second one includes people aged 40-44 years (having 3-20+ years of coding experience), 45-49 (having 5-20+ years of coding experience), 50-59 (having 10-20+ years of coding experience) and 60-70+ (having 20+ years of coding experience). 

There is an assumption. Since we don't know when people aged 60-70+ with 20+ years of coding experience got started, they are included in both the first and the second group.

And the third subsample includes people aged 40+ who has never written code.

<!-- of the same age  40-44 years (having 3-20+ years of coding experience), 45-49 (having 5-20+ years of coding experience), 50-54 (having 1-10 years of coding experience) and 55-70+ (having 1-20 years of coding experience).
 -->

In [ ]:
# 1.  Subset of respondents who started coding after the age of 40 years (group 1)
after_40 = pd.concat([data[data['What is your age (# years)?'].isin(['60-69', '70+']) & \
                data['For how many years have you been writing code and/or programming?'].isin(['1-3 years', '3-5 years', \
                                                                                                '5-10 years', '10-20 years', '20+ years' ])],\
                       data[(data['What is your age (# years)?'] == '50-54') & \
                        data['For how many years have you been writing code and/or programming?'].isin(['1-3 years','3-5 years', \
                                                                                                '5-10 years'])], \
                      data[(data['What is your age (# years)?'] == '55-59') & \
                        data['For how many years have you been writing code and/or programming?'].isin(['1-3 years','3-5 years', \
                                                                                                '5-10 years'])], \
                      data[(data['What is your age (# years)?'] == '40-44') & \
                        (data['For how many years have you been writing code and/or programming?'] == '1-3 years')], \
                       data[(data['What is your age (# years)?'] == '45-49') & \
                        (data['For how many years have you been writing code and/or programming?'].isin(['1-3 years','3-5 years']))]], axis = 0)

# 2. Subsample of respondents who started coding before the age of 40 years (group 2)
before_40 = data[~data.index.isin(after_40.index)]
# before_40 = before_40[~before_40.index.isin(newbies.index)]
remove_exp = data[data['For how many years have you been writing code and/or programming?'].isin(['< 1 years', 'I have never written code'])]
remove_age = data[~data['What is your age (# years)?'].isin(['40-44','45-49','50-54','55-59'])]

# remove_age = data[data['What is your age (# years)?'] == '18-21']

before_40 = before_40[~before_40.index.isin(remove_exp.index)]
before_40 = before_40[~before_40.index.isin(remove_age.index)]
before_40 = pd.concat([before_40, after_40[after_40['What is your age (# years)?'].isin(['60-69', '70+']) & \
                              (after_40['For how many years have you been writing code and/or programming?'] == '20+ years')]])

# 3. Subset of respondents who has never written code 
never_code_after_40 = pd.DataFrame(data[(data['For how many years have you been writing code and/or programming?'] == "I have never written code") & \
                              data['What is your age (# years)?'].isin(['40-44','45-49', '50-54', '55-59', '60-69', '70+'])])

# set(before_40[before_40['What is your age (# years)?'] == '70+']['For how many years have you been writing code and/or programming?'])

In [ ]:
compare_before_after_never = pd.concat([mean_salary_by_age(before_40, 'before the age of 40'), mean_salary_by_age(after_40, 'after the age of 40'), mean_salary_by_age(never_code_after_40, 'never written code')], axis = 1)
compare_before_after_never.plot(kind = 'bar', figsize = (11,5), title = 'Mean salary by age and category')
plt.ylabel('Average yearly compensation, $')
plt.show()

We can see a pattern - at the same age people who started coding after the age of 40 earn less than those who started before this age and more than respondetnts without coding experience. (The group of 70+ years stands out because of an outcome described above). 

This pattern takes place because by the same age these categories of people have different coding experience. And respondents with the same coding experience have almost identical salaries (regardless of the age of start): 

In [ ]:
before_40_short = mean_salary_by_exp(before_40, 'before the age of 40')
after_40_short = mean_salary_by_exp(after_40, 'after the age of 40')
compare_after_before = pd.concat([before_40_short, after_40_short], axis = 1)
compare_after_before.plot(kind = 'bar', title = 'Comparison of salaries ', figsize = (11,5))
plt.ylabel('Average yearly compensation, $')
plt.show()

 ## Comparative analysis of professions by choosen groups 

Let's take a look into the occupational distribution among three choosen categories.  

In [ ]:
before_professions = before_40['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()
after_professions = after_40['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()
never_professions = never_code_after_40['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts()

pie_before = go.Pie(labels=before_professions.index,values=before_professions,name="before",hole=0.6,domain={'x': [0,0.31]})
pie_after = go.Pie(labels=after_professions.index,values=after_professions,name="after",hole=0.6,domain={'x': [0.34,0.65]})
pie_never = go.Pie(labels=never_professions.index,values=after_professions,name="never",hole=0.6,domain={'x': [0.68,0.99]})



layout = dict(title = 'Current roles among the people aged 40+ (by categories)', font=dict(size=10), legend=dict(orientation="v"), autosize=False, width=1000, height=500,
              annotations = [dict(x=0.12, text='before 40', showarrow=False, font=dict(size=14)),
                             dict(x=0.47, text='after 40', showarrow=False, font=dict(size=14)),
                             dict(x=0.83, text='never', showarrow=False, font=dict(size=14))])

fig = dict(data=[pie_before, pie_after, pie_never], layout=layout)

py.iplot(fig)

The diagrams above show that the largest share of unemployed respondents (11,8%) is in group who started coding (or programming) after the age of 40. It can be assumed that it's because there are a lot of people who  have little coding experience.

But the diagrams below show that it's regardless of years of coding experience. The share of currently not employed people among those who started coding after 40 years decreases with experience (from 15% to 7,6% ) but is still 2-3 times higher than this figure among people who started erlier or didn't code at all: 

In [ ]:
after_40_exp_3= after_40[after_40['For how many years have you been writing code and/or programming?'] == '1-3 years']['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
after_40_exp_3.columns = ['Current role', 'Count']

after_40_exp_5 = after_40[after_40['For how many years have you been writing code and/or programming?'] == '3-5 years']['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
after_40_exp_5.columns = ['Current role', 'Count']

after_40_exp_10 = after_40[after_40['For how many years have you been writing code and/or programming?'] == '5-10 years']['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
after_40_exp_10.columns = ['Current role', 'Count']

after_40_exp_20 = after_40[after_40['For how many years have you been writing code and/or programming?'] == '10-20 years']['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
after_40_exp_20.columns = ['Current role', 'Count']

after_40_exp_20_plus = after_40[after_40['For how many years have you been writing code and/or programming?'] == '20+ years']['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts().reset_index()
after_40_exp_20_plus.columns = ['Current role', 'Count']

pie_exp_3 = go.Pie(labels=after_40_exp_3['Current role'],values=after_40_exp_3['Count'],name="1 - 3 years",hole=0.6,domain={'x': [0,0.48], 'y': [0,0.3]})
pie_exp_5 = go.Pie(labels=after_40_exp_5['Current role'],values=after_40_exp_5['Count'],name="3 - 5 years",hole=0.6,domain={'x': [0.5,0.98], 'y': [0,0.3]})
pie_exp_10 = go.Pie(labels=after_40_exp_10['Current role'],values=after_40_exp_10['Count'],name="5 - 10 years",hole=0.6,domain={'x': [0,0.48], 'y': [0.33, 0.65]})
pie_exp_20 = go.Pie(labels=after_40_exp_20['Current role'],values=after_40_exp_20['Count'],name="10 - 20 years",hole=0.6,domain={'x': [0.5, 0.98], 'y': [0.33, 0.65]})
pie_exp_20_plus = go.Pie(labels=after_40_exp_20_plus['Current role'],values=after_40_exp_20_plus['Count'],name="20+ years",hole=0.6, domain={'y': [0.66, 0.98]})


layout = dict(title = 'Current roles among the people who started coding after 40 (by years of experience)', font=dict(size=10), legend=dict(orientation="v"), autosize=False, width=900, height=800,
              annotations = [dict(x=0.18, y=0.15, text='1-3 years', showarrow=False, font=dict(size=14)),
                             dict(x=0.8, y=0.15, text='3-5 years', showarrow=False, font=dict(size=14)),
                             dict(x=0.18, y=0.5, text='5-10 years', showarrow=False, font=dict(size=14)),
                             dict(x=0.81, y=0.5, text='10-20 years', showarrow=False, font=dict(size=14)),
                             dict(y=0.85, text='20+ years', showarrow=False, font=dict(size=14))])

fig = dict(data=[pie_exp_3, pie_exp_5, pie_exp_10, pie_exp_20, pie_exp_20_plus], layout=layout)

py.iplot(fig)

May be the reasons are psychological - there are a lot of people among those who are not afraid of changes (namely start coding in midlife and later) and want to change their professional field and may be take some time out, educated themselves.

And now let's take a look into the difference in income by professions depending on the age of start. The diagram below shows that there is one profession in the data science field where your compensations is regardless of coding/programming skills - DBA/Database Engineer. Salaries of other professions are higher with this skills.  

In [ ]:
mean_sal_before_40_profession_short = mean_salary_by_profession(before_40, 'before 40')
mean_sal_after_40_profession_short = mean_salary_by_profession(after_40, 'after 40')
mean_sal_never_profession_short = mean_salary_by_profession(never_code_after_40, 'after 40')
    
compare_profession = pd.DataFrame(data=[mean_sal_before_40_profession_short, mean_sal_after_40_profession_short, mean_sal_never_profession_short], index = ['before the age of 40', 'after the age of 40', 'never'])

compare_profession.T.plot(kind='bar', figsize = (11,5), title = 'Income by professions depending on the age of start')
plt.ylabel('Average yearly compensation, $')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

In [ ]:
cmap = sns.diverging_palette(5, 250, as_cmap=True)
compare_profession.style.background_gradient(cmap, axis=1).set_precision(0)   

# What can we take away from this analysis?

1. It's just unconscious attitudes, stereotypes that coding/programming is a young man's game. Salary is almost regardless of the age of start and depends only on coding experience. 

2. It's definetely worth to start coding/programming if you want to -  your salary will grow.  

Data science and Machine learning are friendly to all who just work hard and really love what they do.


Thanks for reading!